In [2]:
%load_ext autoreload
%autoreload 2

In [82]:
# 공통 패키지 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import sys

# src 경로 추가
PROJECT_ROOT = os.path.abspath(os.path.join(".."))
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# seaborn 스타일 설정
sns.set(style="whitegrid")

# 경로 및 상수 설정
DATA_DIR = "../data/processed"
OUTPUT_DIR = "../data/modeling"
SEOUL_STATION_PATH = os.path.join(DATA_DIR, "seoul_env_station_mapped.csv")
OUTPUT_IMG = "../outputs/image"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# KMeans 설정
KMEANS_MODE = "auto" # auto, manual
KMEANS_MANUAL_K = 5

# XGBoost 설정
XGB_N_ESTIMATORS = 100

# MCLP 설정
COVERAGE_RADIUS = 0.005
FACILITY_LIMIT = 30

# 평가 함수 import
from evaluation.baseline_evaluator import (
    evaluate_existing_stations,
    evaluate_random_installation,
    evaluate_cluster_centers,
    evaluate_mclp_result
)

# MCLP + 시나리오

현재 충전소는 그대로 유지

예상 수요 증가를 반영하여 전체 predicted_demand_score를 scaling

각 베이스라인 (랜덤, 클러스터, MCLP 등)은 이 증가된 수요를 기준으로:

추가 충전소를 설치하고

전체 충전소(기존 + 추가)가 얼마나 커버하는지 평가

참고자료1 : https://n.news.naver.com/mnews/article/015/0005137028?sid=100
참고자료2 : https://news.seoul.go.kr/env/archives/558886
참고자료3 : https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1257
- 2024 국내 전기차 운행률 2.6%
- 2030 정부 전기차 보급률 50%
- 2026 서울시 전기차 전환 10%

In [83]:
# 실험 목적 상위 10% 격자 수 추출

# 데이터 로드
future_demand = pd.read_csv("../data/processed/future_demand_gridded_2.csv")

# 상위 10% 수요 격자 추출
demand_k = int(len(future_demand) * 0.10)
top_k = future_demand.nlargest(demand_k, 'final_score').copy()
top_k_ids = set(top_k['grid_id'])

print(f"상위 10% 격자 수 (demand_k): {demand_k}")

상위 10% 격자 수 (demand_k): 208


In [108]:
import pandas as pd

# 데이터 로드
all_df = pd.read_csv("../data/processed/grid_features.csv")  # 전체 격자 (demand_score 포함)
stations = pd.read_csv("../data/processed/charging_stations_seoul_gridded.csv")  # 충전소 (grid_id 포함)

# 전체 격자 수
total_grids = all_df['grid_id'].nunique()

# 설치된 격자 수
installed_grids = stations['grid_id'].nunique()

# 설치 비율
install_ratio = installed_grids / total_grids * 100

# 수요 커버율 계산
covered = all_df[all_df['grid_id'].isin(stations['grid_id'])]
covered_demand = covered['demand_score'].sum()
total_demand = all_df['demand_score'].sum()
coverage_rate = covered_demand / total_demand * 100

# 결과 출력
print("기존 충전소 기반 분석 결과")
print(f"- 전체 서울 격자 수: {total_grids}")
print(f"- 설치 격자 수: {installed_grids}")
print(f"- 서울 전체 기준 설치 비율: {install_ratio:.2f}%")
print(f"- 수요 기준 커버율: {coverage_rate:.2f}%")

기존 충전소 기반 분석 결과
- 전체 서울 격자 수: 6030
- 설치 격자 수: 1753
- 서울 전체 기준 설치 비율: 29.07%
- 수요 기준 커버율: 96.18%


# K-means

In [84]:
from modeling.kmeans_model import run_kmeans

# 데이터 로드
grid = pd.read_csv(f"{DATA_DIR}/grid_system_processed.csv")

# 수요가 가장 높은 클러스터만 추출
grid , used_k = run_kmeans(
    grid,
    mode=KMEANS_MODE,
    # manual_k=KMEANS_MANUAL_K,
    return_top_cluster_only=True
)

Finding optimal k: 100%|██████████| 9/9 [00:00<00:00, 24.44it/s]

[AUTO MODE] 최적 k = 2
Inertia by k: {2: 394878040.6060905, 3: 182752136.79175264, 4: 111630868.37625775, 5: 83130803.13068569, 6: 65292788.3815513, 7: 54917904.7908943, 8: 48181642.39693806, 9: 36633734.731904596, 10: 32347245.131689377}

[Cluster별 평균 수요]
cluster
1.0    1360.575095
0.0      57.366915
Name: demand_score, dtype: float64

[필터링] 수요가 가장 높은 클러스터 (cluster=1.0)만 반환됨.


KMeans로 수요 밀집 지역을 선별한 후,

해당 지역의 feature만 따로 정리해 저장하는 전처리 단계

→ 이후 모델(XGBoost, MCLP)에서 이 subset만 사용할 수 있게 함.

In [95]:
# 필요한 컬럼만 유지 (grid: KMeans 결과)
grid = grid[['grid_id', 'center_lat', 'center_lon', 'cluster']]

# 원본 feature 로드
features_all = pd.read_csv(f"{DATA_DIR}/grid_features.csv")

# 병합 전: features_all 좌표 제거 (혼동 방지)
features_all = features_all.drop(columns=['center_lat', 'center_lon'], errors='ignore')

# 병합
features = features_all.merge(grid, on='grid_id', how='inner')

# cluster 정수형 처리
features['cluster'] = features['cluster'].astype(int)

# 저장 전 좌표 정리: 이미 이름이 정돈돼 있음
# → center_lat, center_lon 이 grid 기준으로만 존재
features = features.loc[:, ~features.columns.duplicated()]  # 혹시 모를 중복 제거

# 저장
features.to_csv(f"{OUTPUT_DIR}/kmeans_grid_features.csv", index=False)
print("좌표 컬럼 중복 없이 정리 및 저장 완료")

# 확인
print(f"사용 가능한 feature 컬럼: {features.columns.tolist()}")

좌표 컬럼 중복 없이 정리 및 저장 완료
사용 가능한 feature 컬럼: ['grid_id', 'demand_score', 'supply_score', 'commercial_count', 'station_count', 'supply_demand_ratio', 'population_density', 'accessibility_score', 'transport_score', 'center_lat', 'center_lon', 'cluster']


In [96]:
import pandas as pd

# 데이터 로드
all_df = pd.read_csv(f"{DATA_DIR}/grid_features.csv")  # 전체 서울 격자
kmeans_df = pd.read_csv(f"{OUTPUT_DIR}/kmeans_grid_features.csv")  # 클러스터 기반 격자

# 전체 서울 격자 수
total_grids = all_df['grid_id'].nunique()

# 클러스터 선택된 격자 수
selected_grids = kmeans_df['grid_id'].nunique()

# 서울 전체 기준 설치 비율
install_ratio = selected_grids / total_grids * 100

# 수요 기준 커버율 (전체 수요 중 클러스터 격자의 수요가 차지하는 비중)
total_demand = all_df['demand_score'].sum()
covered_demand = kmeans_df['demand_score'].sum()
coverage_rate = covered_demand / total_demand * 100

# 출력
print(f"클러스터링 기반 분석 결과")
print(f"- 전체 서울 격자 수: {total_grids}")
print(f"- 클러스터 격자 수: {selected_grids}")
print(f"- 서울 전체 기준 설치 비율: {install_ratio:.2f}%")
print(f"- 수요 기준 커버율: {coverage_rate:.2f}%")

클러스터링 기반 분석 결과
- 전체 서울 격자 수: 6030
- 클러스터 격자 수: 526
- 서울 전체 기준 설치 비율: 8.72%
- 수요 기준 커버율: 69.39%


In [97]:
# 선택시항

from utils.inspect_dataframe import inspect_dataframe
from visualization.map_visualizer import visualize_cluster_map

# 1. kmeans_grid_features.csv 특징
inspect_dataframe(features, name="features")

# 2. kmeans 시각화 - 서울 전체 (회색) - 수요 (빨간색)
visualize_cluster_map(
    all_features_path=f"{DATA_DIR}/grid_features.csv",
    filtered_features_path=f"{OUTPUT_DIR}/kmeans_grid_features.csv",
    output_path=f"../outputs/maps/kmeans_visualization.html"
)

🧾 입력 데이터 이름: features
🔢 shape: (526, 12)
🔑 컬럼 목록:
['grid_id', 'demand_score', 'supply_score', 'commercial_count', 'station_count', 'supply_demand_ratio', 'population_density', 'accessibility_score', 'transport_score', 'center_lat', 'center_lon', 'cluster']

🧾 예시 5개:
        grid_id  demand_score  supply_score  commercial_count  station_count  \
0  GRID_011_036        1372.5      27.15053               200              1   
1  GRID_011_037         911.5      27.15053               200              0   
2  GRID_012_035         883.0      27.15053               200              1   
3  GRID_012_036         952.5      27.15053               200              0   
4  GRID_014_033         946.5      27.15053               200              0   

   supply_demand_ratio  population_density  accessibility_score  \
0            50.551499                2400                  0.0   
1            33.572089                2400                  0.0   
2            32.522385                2400         

# XGBoost

In [98]:
from modeling.xgboost_model import train_and_predict

# 학습 feature
selected_features = [
    'supply_score',
    'station_count',
    'commercial_count',
    'supply_demand_ratio',
    'population_density',
    'accessibility_score',
    'transport_score',
    'cluster'
]

# 범주형 처리
features['cluster'] = features['cluster'].astype('category')

# XGBoost 학습 및 예측
features_with_pred, metrics, model = train_and_predict(
    df=features,
    features=selected_features,
    label='demand_score',
    n_estimators=XGB_N_ESTIMATORS,
    verbose=True
)

# 저장
features_with_pred.to_csv(f"{OUTPUT_DIR}/xgboost_grid_features.csv", index=False)


XGBoost 성능:
MAE: 50.72
RMSE: 81.40
R²: 0.9826


# MCLP

In [109]:
from modeling.mclp_model import solve_mclp

# 1. 설정
COVERAGE_RADIUS_KM = 0.55  # 반경 550m

# 2. MCLP 실행
final_df, final_summary, _ = solve_mclp(
    df=features_with_pred,
    coverage_radius=COVERAGE_RADIUS_KM,  # 단위: km로 통일
    facility_limit=demand_k,
    demand_column='demand_score',
    verbose=True
)

설치지 수: 208개
커버 수요: 713,497.50 / 총 수요: 715,662.50
커버율: 99.70%


In [100]:
# 3. 결과 저장
final_df.to_csv(f"{OUTPUT_DIR}/mclp_grid_features.csv", index=False)                 # 전체 결과
final_df[final_df['selected'] == 1].to_csv(f"{OUTPUT_DIR}/mclp_selected_grid_features.csv", index=False)  # 선택된 설치 위치만 저장

In [101]:
# 선택사항 - mclp 후 지역 시각화

from visualization.map_visualizer import visualize_selected_sites_map

visualize_selected_sites_map(
    features_path=f"{OUTPUT_DIR}/mclp_grid_features.csv",
    output_path=f"../outputs/maps/mclp_result_map.html"
)

설치지 시각화 저장 완료: ../outputs/maps/mclp_result_map.html


In [102]:
import pandas as pd

# MCLP 결과 데이터 로드
df = pd.read_csv(f"{OUTPUT_DIR}/mclp_grid_features.csv")

# 설치된 격자 수
selected_count = df[df['selected'] == 1].shape[0]

# 전체 격자 수
total_count = df.shape[0]

# 비율 계산
ratio = selected_count / total_count * 100

print(f"설치 격자 수: {selected_count}")
print(f"전체 격자 수: {total_count}")
print(f"설치 비율: {ratio:.2f}%")


설치 격자 수: 208
전체 격자 수: 526
설치 비율: 39.54%


In [103]:
# 전체 서울 격자 수
all_df = pd.read_csv("../data/processed/grid_features.csv")
all_total_count = all_df['grid_id'].nunique()

# 설치 비율 (서울 전체 기준)
real_ratio = selected_count / all_total_count * 100

print(f"서울 전체 기준 설치 비율: {real_ratio:.2f}%")

서울 전체 기준 설치 비율: 3.45%


# 평가

## 설치 방식 비교 실험

### 1. 실험 목적
전기차 충전소 입지 선정 방식에 따라 **미래 수요 상위 지역을 얼마나 잘 커버하는지**를 비교한다.  
수요 기반 알고리즘(MCLP, 클러스터링 등)이 **기존 설치 방식이나 무작위 설치 대비 얼마나 효율적인지** 확인하는 것이 목적이다.

---

### 2. 실험 방식

- **수요 기준**: `future_demand_gridded.csv`의 adjusted_ev_demand 컬럼 기준
- **상위 격자 수**: 전체 격자 중 상위 20%인 `demand_k = 47`개
- **설치 방식별 선정 방식**:
  - **기존 충전소 전체**: 서울시의 모든 기존 충전소 설치 격자
  - **랜덤 설치**: 기존 충전소 중 무작위 47개 선택
  - **클러스터 기반 설치**: KMeans 클러스터 격자 중 `demand_score` 상위 47개
  - **MCLP 추천 설치**: KMeans → XGBoost → MCLP 로 구성된 복합 전략

---

### 3. 평가 지표

- **포함된 격자 수**: 상위 47개 수요 격자 중 설치된 격자의 수
- **커버 수요**: 해당 격자들이 커버한 수요 총합
- **커버율 (%)**: `(커버 수요 / 전체 상위 수요) * 100`
- **설치 효율**: `커버 수요 / 설치 격자 수`

---

### 4. 실험 결과

| 설치 방식             | 포함 격자 수 | 커버 수요     | 전체 수요     | 커버율 (%) | 설치 효율 |
|----------------------|--------------|---------------|---------------|-------------|------------|
| 기존 충전소 전체      | 33 / 47      | 275,653.50    | 324,565.50    | 84.93       | 157.25     |
| 랜덤 설치            | 0 / 47       | 0.00          | 324,565.50    | 0.00        | 0.00       |
| 클러스터 기반 설치   | 1 / 47       | 6,103.50      | 324,565.50    | 1.88        | 129.86     |
| MCLP 추천 설치       | 4 / 47       | 34,884.00     | 324,565.50    | 10.75       | 742.21     |

---

### 5. 분석 요약

- **기존 충전소 전체**는 높은 커버율을 보이지만 설치 효율은 낮다.
- **랜덤 설치**는 커버 수요가 전무하여 가장 비효율적이다.
- **클러스터 기반 설치**는 예측 수요는 높았지만, 상위 수요 지역과 겹침이 적어 실효성이 낮다.
- **MCLP 추천 설치**는 상대적으로 적은 수의 설치만으로도 높은 효율을 보이며, 전략적 입지 선정의 효과를 입증하였다.

In [104]:
import pandas as pd

# 1. 데이터 로드
stations = pd.read_csv("../data/processed/charging_stations_seoul_gridded.csv")
future_demand = pd.read_csv("../data/processed/future_demand_gridded_2.csv")
grid_features = pd.read_csv("../data/processed/grid_features.csv")                 
kmeans_features = pd.read_csv("../data/modeling/kmeans_grid_features.csv")         # 클러스터 기반
mclp_features = pd.read_csv("../data/modeling/mclp_selected_grid_features.csv")    # MCLP 결과

# 2. 전체 격자 중 adjusted_ev_demand 기준 상위 10% 추출
demand_k = int(len(future_demand) * 0.10)
top_k = future_demand.nlargest(demand_k, 'adjusted_ev_demand').copy()
top_k_ids = set(top_k['grid_id'])

print(f"상위 10% 격자 수 (demand_k): {demand_k}")

# 3. 설치 방식별 격자 선택

# 3-1. 기존 충전소 전체
existing_grids = set(stations['grid_id'])

# 3-2. 랜덤 설치: 기존 충전소가 설치된 격자 중 demand_k개 무작위 선택
random_grids = set(stations.sample(n=demand_k, random_state=42)['grid_id'])

# 3-3. 클러스터 기반 설치: KMeans 클러스터 내 격자들 중 demand_score 기준 상위 demand_k개
cluster_sorted = kmeans_features.sort_values('demand_score', ascending=False)
cluster_grids = set(cluster_sorted['grid_id'].dropna().head(demand_k))

# 3-4. MCLP 추천 설치: selected == 1로 표시된 격자 모두 사용
mclp_grids = set(mclp_features[mclp_features['selected'] == 1]['grid_id'])

# 4. 평가 함수
def evaluate_grid_overlap(top_k_df, selected_grids, label):
    matched = top_k_ids & selected_grids
    matched_count = len(matched)
    matched_demand = top_k_df[top_k_df['grid_id'].isin(matched)]['adjusted_ev_demand'].sum()
    total_demand = top_k_df['adjusted_ev_demand'].sum()
    coverage_rate = matched_demand / total_demand * 100
    efficiency = matched_demand / len(selected_grids) if selected_grids else 0

    print(f"\n[{label}]")
    print(f"- 포함된 격자 수: {matched_count} / 상위 {len(top_k_df)}개")
    print(f"- 커버 수요: {matched_demand:,.2f}")
    print(f"- 전체 수요: {total_demand:,.2f}")
    print(f"- 커버율: {coverage_rate:.2f}%")
    print(f"- 설치 1개당 커버 수요 (효율): {efficiency:,.2f}")

# 5. 평가 실행
evaluate_grid_overlap(top_k, existing_grids, "기존 충전소 전체")
evaluate_grid_overlap(top_k, random_grids, "랜덤 설치")
evaluate_grid_overlap(top_k, cluster_grids, "클러스터 기반 설치")
evaluate_grid_overlap(top_k, mclp_grids, "MCLP 추천 설치")

상위 10% 격자 수 (demand_k): 208

[기존 충전소 전체]
- 포함된 격자 수: 185 / 상위 208개
- 커버 수요: 236,210.50
- 전체 수요: 250,031.30
- 커버율: 94.47%
- 설치 1개당 커버 수요 (효율): 134.75

[랜덤 설치]
- 포함된 격자 수: 17 / 상위 208개
- 커버 수요: 20,161.80
- 전체 수요: 250,031.30
- 커버율: 8.06%
- 설치 1개당 커버 수요 (효율): 108.40

[클러스터 기반 설치]
- 포함된 격자 수: 32 / 상위 208개
- 커버 수요: 47,495.40
- 전체 수요: 250,031.30
- 커버율: 19.00%
- 설치 1개당 커버 수요 (효율): 228.34

[MCLP 추천 설치]
- 포함된 격자 수: 33 / 상위 208개
- 커버 수요: 42,180.00
- 전체 수요: 250,031.30
- 커버율: 16.87%
- 설치 1개당 커버 수요 (효율): 202.79


# XX (기타)

In [14]:
import pandas as pd

# 파일 경로 지정
paths = {
    "future_demand": "../data/processed/future_demand_gridded_2.csv",
    "grid_features": "../data/processed/grid_features.csv",
    "kmeans_features": "../data/modeling/kmeans_grid_features.csv",
    "mclp_features": "../data/modeling/mclp_selected_grid_features.csv",
    "charging_stations": "../data/processed/charging_stations_seoul_gridded.csv"
}

# 결과 출력
for name, path in paths.items():
    df = pd.read_csv(path)
    n = df['grid_id'].nunique()
    print(f"{name} → grid_id 개수: {n}")

future_demand → grid_id 개수: 397
grid_features → grid_id 개수: 6030
kmeans_features → grid_id 개수: 526
mclp_features → grid_id 개수: 89
charging_stations → grid_id 개수: 1753


In [15]:
import pandas as pd

# 데이터 불러오기
future = pd.read_csv("../data/processed/future_demand_gridded_2.csv")
grid = pd.read_csv("../data/processed/grid_features.csv")  # center_lat, center_lon 포함

# 병합 (좌표 추가)
merged = future.merge(grid[['grid_id', 'center_lat', 'center_lon']], on='grid_id', how='left')

# 확인
print(merged.head())

# 저장 (선택)
merged.to_csv("../data/processed/future_demand_gridded_2_with_coords.csv", index=False)

        grid_id  demand_score  adjusted_ev_demand  log_future  norm_current  \
0  GRID_010_036         425.0               474.8    6.164998      0.127954   
1  GRID_010_037         118.5               587.5    6.377577      0.035677   
2  GRID_011_036        1372.5              1306.1    7.175566      0.413217   
3  GRID_011_038         328.0               759.5    6.633976      0.098751   
4  GRID_012_033          79.5                 0.1    0.095310      0.023935   

   norm_log_future  final_score  center_lat  center_lon  
0         0.694958     0.354756    37.44725    126.9044  
1         0.718921     0.308974    37.44725    126.9100  
2         0.808875     0.571480    37.45175    126.9044  
3         0.747824     0.358380    37.45175    126.9156  
4         0.010744     0.018659    37.45625    126.8876  


In [16]:
import pandas as pd
import folium

# 1. 파일 경로 및 컬럼 정보 정의
datasets = {
    "future_demand": {
        "path": "../data/processed/future_demand_gridded_2_with_coords.csv",
        "lat": "center_lat",
        "lon": "center_lon",
        "color": "green"
    },
    "grid_features": {
        "path": "../data/processed/grid_features.csv",
        "lat": "center_lat",
        "lon": "center_lon",
        "color": "gray"
    },
    "kmeans_features": {
        "path": "../data/modeling/kmeans_grid_features.csv",
        "lat": "center_lat",
        "lon": "center_lon",
        "color": "blue"
    },
    "mclp_features": {
        "path": "../data/modeling/mclp_selected_grid_features.csv",
        "lat": "center_lat",
        "lon": "center_lon",
        "color": "red"
    },
    "charging_stations": {
        "path": "../data/processed/charging_stations_seoul_gridded.csv",
        "lat": "위도",
        "lon": "경도",
        "color": "purple"
    }
}

# 2. 지도 초기화
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 3. 각 데이터셋을 지도에 추가
for name, info in datasets.items():
    df = pd.read_csv(info["path"])
    lat_col = info["lat"]
    lon_col = info["lon"]
    color = info["color"]

    layer = folium.FeatureGroup(name=name, show=True)

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row[lat_col], row[lon_col]],
            radius=3,
            color=color,
            fill=True,
            fill_opacity=0.5,
            popup=f"{name}: {row.get('grid_id', 'N/A')}"
        ).add_to(layer)

    layer.add_to(m)

# 4. 레이어 컨트롤 추가 및 저장
folium.LayerControl().add_to(m)
m.save("../outputs/maps/all_grids_combined_map.html")
print("✅ 지도 저장 완료: ../outputs/maps/all_grids_combined_map.html")

✅ 지도 저장 완료: ../outputs/maps/all_grids_combined_map.html


In [105]:
# 1. 평가 대상 불러오기
future_demand = pd.read_csv("../data/processed/future_demand_gridded_2.csv")
stations = pd.read_csv("../data/processed/charging_stations_seoul_gridded.csv")
kmeans = pd.read_csv("../data/modeling/kmeans_grid_features.csv")
mclp = pd.read_csv("../data/modeling/mclp_selected_grid_features.csv")

# 2. 상위 20% 격자 추출
demand_k = int(len(future_demand) * 0.10)
top_k = future_demand.nlargest(demand_k, 'final_score')
top_k_ids = set(top_k['grid_id'])

# 3. 전략별 설치 격자 추출
strategy_grids = {
    "기존 충전소 전체": set(stations['grid_id']),
    "랜덤 설치": set(stations.sample(n=demand_k, random_state=42)['grid_id']),
    "클러스터 기반 설치": set(kmeans.sort_values('demand_score', ascending=False)['grid_id'].dropna().head(demand_k)),
    "MCLP 추천 설치": set(mclp[mclp['selected'] == 1]['grid_id'])
}

# 4. 공통 평가 함수 (모두 격자 기준)
def evaluate_strategy(name, selected_grids):
    matched = top_k_ids & selected_grids
    matched_demand = top_k[top_k['grid_id'].isin(matched)]['adjusted_ev_demand'].sum()
    total_demand = top_k['adjusted_ev_demand'].sum()
    efficiency = matched_demand / len(selected_grids) if selected_grids else 0

    print(f"\n[{name}]")
    print(f"- 설치 수: {len(selected_grids)}")
    print(f"- 커버 격자 수: {len(matched)} / {len(top_k_ids)}")
    print(f"- 커버 수요: {matched_demand:.2f} / 전체 수요: {total_demand:.2f}")
    print(f"- 커버율: {matched_demand / total_demand * 100:.2f}%")
    print(f"- 설치 효율 (1개당 수요): {efficiency:.2f}")

# 5. 실행
for name, grid_ids in strategy_grids.items():
    evaluate_strategy(name, grid_ids)



[기존 충전소 전체]
- 설치 수: 1753
- 커버 격자 수: 183 / 188
- 커버 수요: 191075.20 / 전체 수요: 196501.50
- 커버율: 97.24%
- 설치 효율 (1개당 수요): 109.00

[랜덤 설치]
- 설치 수: 186
- 커버 격자 수: 18 / 188
- 커버 수요: 14040.30 / 전체 수요: 196501.50
- 커버율: 7.15%
- 설치 효율 (1개당 수요): 75.49

[클러스터 기반 설치]
- 설치 수: 208
- 커버 격자 수: 69 / 188
- 커버 수요: 62129.10 / 전체 수요: 196501.50
- 커버율: 31.62%
- 설치 효율 (1개당 수요): 298.70

[MCLP 추천 설치]
- 설치 수: 208
- 커버 격자 수: 56 / 188
- 커버 수요: 54157.50 / 전체 수요: 196501.50
- 커버율: 27.56%
- 설치 효율 (1개당 수요): 260.37


In [79]:
# 평가 기준 (top_k)
top_k_ids = set(top_k['grid_id'])

# 예: MCLP 결과
mclp = pd.read_csv("../data/modeling/mclp_selected_grid_features.csv")
selected_ids = set(mclp[mclp['selected'] == 1]['grid_id'])

# 평가 대상에서 실제로 존재하는 grid_id
actual_ids_in_mclp = top_k_ids & set(mclp['grid_id'])

print("top_k grid_id 수:", len(top_k_ids))  # 89
print("MCLP 파일 내 포함된 grid_id 수:", len(actual_ids_in_mclp))  # 76? 77?
print("누락된 grid_id 목록:", top_k_ids - set(mclp['grid_id']))

top_k grid_id 수: 188
MCLP 파일 내 포함된 grid_id 수: 56
누락된 grid_id 목록: {'GRID_018_040', 'GRID_023_073', 'GRID_022_075', 'GRID_033_039', 'GRID_017_033', 'GRID_034_039', 'GRID_029_076', 'GRID_029_052', 'GRID_032_063', 'GRID_020_051', 'GRID_023_058', 'GRID_023_032', 'GRID_036_026', 'GRID_030_046', 'GRID_038_050', 'GRID_039_066', 'GRID_048_038', 'GRID_042_067', 'GRID_018_076', 'GRID_024_058', 'GRID_036_052', 'GRID_032_066', 'GRID_029_077', 'GRID_033_079', 'GRID_029_054', 'GRID_024_043', 'GRID_022_033', 'GRID_028_048', 'GRID_026_062', 'GRID_022_060', 'GRID_019_044', 'GRID_027_033', 'GRID_019_037', 'GRID_021_040', 'GRID_022_040', 'GRID_022_035', 'GRID_028_035', 'GRID_059_060', 'GRID_037_052', 'GRID_035_067', 'GRID_035_027', 'GRID_034_045', 'GRID_056_065', 'GRID_030_030', 'GRID_046_057', 'GRID_035_049', 'GRID_015_042', 'GRID_026_030', 'GRID_038_018', 'GRID_023_063', 'GRID_054_057', 'GRID_037_036', 'GRID_018_049', 'GRID_024_074', 'GRID_042_063', 'GRID_018_050', 'GRID_028_034', 'GRID_029_068', 'GRID_